# Advanced Querying Mongo

Importing libraries and setting up connection

In [50]:
from pymongo import MongoClient

In [51]:
client = MongoClient()

In [52]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [53]:
db=client.get_database("companies")

In [54]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [55]:
comp = db.companies.find({})

In [56]:
len(list(comp))

18801

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [90]:
# Your Code
res = db.companies.find({"name":"Babelgum"})
res = list(res)

In [91]:
res[0]["name"]

'Babelgum'

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [123]:
# Your Code
filter_query={"number_of_employees":{"$gt":5000}}
project = {"name":1}
res = db.companies.find(filter_query,project).sort([("number_of_employees",-1)]).limit(20)
list(res)[:4]

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'), 'name': 'Siemens'},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'), 'name': 'IBM'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'), 'name': 'Toyota'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'), 'name': 'PayPal'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [95]:
# Your Code
filter_query={"founded_year":{"$gte":2000, "$lte":2005}}
project={"name":1,"founded_year":1}
res=db.companies.find(filter_query,project)
list(res)[:4]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d96'),
  'name': 'Gizmoz',
  'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [96]:
# Your Code
filter_query={"ipo.valuation_amount":{"$gt":100000000}}
project={"name":1,"ipo":1}
res=db.companies.find(filter_query,project)
list(res)[:2]

[{'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [100]:
# Your Code
filter_query={"number_of_employees":{"$lt":1000},"founded_year":{"$lte":2005}}
project = {"name":1}
res = db.companies.find(filter_query,project).sort([("number_of_employees",-1)]).limit(10)
print(list(res[:4]))

[{'_id': ObjectId('52cdef7d4bab8bd675298933'), 'name': 'Infinera Corporation'}, {'_id': ObjectId('52cdef7c4bab8bd675297e10'), 'name': 'Box'}, {'_id': ObjectId('52cdef7e4bab8bd67529ac95'), 'name': 'NorthPoint Communications Group'}, {'_id': ObjectId('52cdef7f4bab8bd67529be17'), 'name': '888 Holdings'}]


### 6. All the companies that don't include the `partners` field.

In [86]:
# Your Code +
filter_query={"partners":{"$exists":"false"}}
project = {"name":1}
res = list(db.companies.find(filter_query,project))
print(len(res))
res[:4]

18801


[{'_id': ObjectId('52cdef7c4bab8bd675297d8f'), 'name': 'Omnidrive'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8b'), 'name': 'AdventNet'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d92'), 'name': 'Flektor'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [85]:
# Your Code
filter_query={"category_code":{"$type":"null"}}
project = {"name":1}
res = list(db.companies.find(filter_query,project))
print(len(res))
res[:4]

2751


[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'), 'name': 'Collective'},
 {'_id': ObjectId('52cdef7c4bab8bd675298225'), 'name': 'Snimmer'},
 {'_id': ObjectId('52cdef7c4bab8bd675298226'), 'name': 'KoolIM'},
 {'_id': ObjectId('52cdef7c4bab8bd675298261'), 'name': 'Level9 Media'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [112]:
# Your Code
filter_query={"number_of_employees":{"$gte":100,"$lt":1000}}
project = {"name":1,"number_of_employees":1}
res = list(db.companies.find(filter_query,project))
print(len(res))
res[:4]

917


[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600},
 {'_id': ObjectId('52cdef7c4bab8bd675297da7'),
  'name': 'AddThis',
  'number_of_employees': 120},
 {'_id': ObjectId('52cdef7c4bab8bd675297da8'),
  'name': 'OpenX',
  'number_of_employees': 305},
 {'_id': ObjectId('52cdef7c4bab8bd675297db5'),
  'name': 'LifeLock',
  'number_of_employees': 644}]

### 9. Order all the companies by their IPO price in a descending order.

In [136]:
# Your Code
filter_query={}
project = {"name":1}
res = db.companies.find(filter_query,project).sort("ipo.valuation_amount:",-1)
list(res)[:4]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8f'), 'name': 'Omnidrive'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8b'), 'name': 'AdventNet'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d92'), 'name': 'Flektor'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [135]:
# Your Code
filter_query={}
project = {"name":1}
res = db.companies.find(filter_query,project).sort("number_of_employees",-1).limit(10)
list(res)[:4]

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'), 'name': 'Siemens'},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'), 'name': 'IBM'},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'), 'name': 'Toyota'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'), 'name': 'PayPal'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [117]:
# Your Code
filter_query={"founded_month":{"$gt":6}}
project = {"name":1}
res = list(db.companies.find(filter_query,project).limit(1000))
res[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8f'), 'name': 'Omnidrive'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'), 'name': 'Zoho'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'name': 'Wetpaint'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'), 'name': 'Digg'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9d'), 'name': 'Joost'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [119]:
# Your Code
filter_query={"acquisition.price_amount":{"$gt":10000000},"founded_year":{"$lte":2000}}
project = {"name":1}
res = list(db.companies.find(filter_query,project))
print(len(res))
print(res[:4])

246
[{'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'}, {'_id': ObjectId('52cdef7c4bab8bd675297deb'), 'name': 'SideStep'}, {'_id': ObjectId('52cdef7c4bab8bd675297e2c'), 'name': 'Recipezaar'}, {'_id': ObjectId('52cdef7c4bab8bd675297e97'), 'name': 'Snapfish'}]


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [143]:
# Your Code
filter_query={"acquisition.acquired_year":{"$gt":2010}}
project = {"name":1,"acquisition":1}
res = db.companies.find(filter_query,project).sort("acquisition.price_amount",-1)
list(res)[:1]

[{'_id': ObjectId('52cdef7c4bab8bd675298876'),
  'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [139]:
# Your Code
filter_query={}
project = {"name":1,"founded_year":1}
res = db.companies.find(filter_query,project).sort("founded_year",1)
list(res)[:4]

[{'_id': ObjectId('52cdef7c4bab8bd675297d92'),
  'name': 'Flektor',
  'founded_year': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297d99'),
  'name': 'Lala',
  'founded_year': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297da6'),
  'name': 'SpinVox',
  'founded_year': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297db0'),
  'name': 'Info',
  'founded_year': None}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [144]:
# Your Code
filter_query={"founded_day":{"$lte":7}}
project = {"name":1}
res = db.companies.find(filter_query,project).sort("acquisition.price_amount",-1).limit(10)
list(res)[:5]

[{'_id': ObjectId('52cdef7d4bab8bd6752989a1'), 'name': 'Netscape'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'), 'name': 'PayPal'},
 {'_id': ObjectId('52cdef7c4bab8bd675297efe'), 'name': 'Zappos'},
 {'_id': ObjectId('52cdef7c4bab8bd675297f0c'), 'name': 'Alibaba'},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [147]:
# Your Code
filter_query={"number_of_employees":{"$gt":4000},"category_code":"web"}
project = {"name":1}
res = db.companies.find(filter_query,project).sort("number_of_employees",1)
list(res)[:4]

[{'_id': ObjectId('52cdef7c4bab8bd67529822a'), 'name': 'Expedia'},
 {'_id': ObjectId('52cdef7c4bab8bd675297e96'), 'name': 'AOL'},
 {'_id': ObjectId('52cdef7c4bab8bd675297ea4'), 'name': 'Webkinz'},
 {'_id': ObjectId('52cdef7c4bab8bd675297fcb'), 'name': 'Rakuten'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [148]:
# Your Code
filter_query={"acquisition.price_amount":{"$gt":10000000},"acquisition.price_currency_code":"EUR"}
project = {"name":1}
res = list(db.companies.find(filter_query,project))
print(len(res))
print(res[:4])


7
[{'_id': ObjectId('52cdef7c4bab8bd675297f02'), 'name': 'ZYB'}, {'_id': ObjectId('52cdef7d4bab8bd675298bf3'), 'name': 'Apertio'}, {'_id': ObjectId('52cdef7d4bab8bd675298f47'), 'name': 'Greenfield Online'}, {'_id': ObjectId('52cdef7e4bab8bd67529a536'), 'name': 'Webedia'}]


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [149]:
# Your Code
filter_query={"acquisition.acquired_month":{"$lte":3}}
project = {"name":1,"acquisition":1}
res = db.companies.find(filter_query,project).limit(10)
list(res)[:1]

[{'_id': ObjectId('52cdef7c4bab8bd675297db4'),
  'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   'acquired_day': None,
   'acquiring_company': {'name': 'Nielsen', 'permalink': 'nielsen'}}}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code